In [5]:
import json
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [68]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [49]:
# def get_completion(prompt, model="gpt-3.5-turbo"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.Completion.create(
#         engine="text-davinci-002",  # choose the engine you want to use
#         prompt=prompt,
#         max_tokens=150
#     )

#     # Extract the model's reply from the response
#     model_reply = response['choices'][0]['text']

#     # Create a dictionary to store the result
#     result = {"input_prompt": prompt, "model_reply": model_reply}

#     return result

In [ ]:
# import openai
# import json

# def get_completion(prompt):
#     # Set your OpenAI API key
#     openai.api_key = 'your-api-key'

#     # Make a request to the OpenAI API
#     response = openai.Completion.create(
#         engine="text-davinci-002",  # choose the engine you want to use
#         prompt=prompt,
#         max_tokens=150
#     )

#     # Extract the model's reply from the response
#     model_reply = response['choices'][0]['text']

#     # Create a dictionary to store the result
#     result = {"input_prompt": prompt, "model_reply": model_reply}

#     return result

# # Example usage
# prompt_text = "Translate the following English text to French:"

# result_dict = get_completion(prompt_text)

# # Specify the file path
# json_file_path = "output.json"

# # Write the result dictionary to the JSON file
# with open(json_file_path, 'w') as json_file:
 #   json.dump(result_dict, json_file, indent=2)


In [113]:
import openai
import os
human_text = "Build me an for the breast-cancer-wisconsin  dataset with default max_depth = 3, splitter = random and criterion as log loss and target variable 'Class' and print the evaluation metrics for all"

prompt = f"""
read the prompt and create a dictionary. include filename and append the dataset filename with .csv extension. create  a dictionary named param (which would be the parameters of the model) and write all the parameters asked by the user in the datatype of what the function requires. 
include 'target_variable' as mentioned in the prompt. and 'split' should be 0.2 unless some other value is specified. an example for svm model might look like this (with curly brackets instead of sqauare brackets):
[
    
    "filename" : "breast-cancer-wisconsin.csv",
    "model_name" : "svm",
    "param": [
        "kernel": "linear"
    ],
    "target_variable": "Class",
    "split" : 0.2
]
text = ```{human_text}```
"""


json_objects = get_completion(prompt)
print(json_objects)


{
    "filename": "breast-cancer-wisconsin.csv",
    "model_name": "decision_tree",
    "param": {
        "max_depth": 3,
        "splitter": "random",
        "criterion": "log_loss"
    },
    "target_variable": "Class",
    "split": 0.2
}


In [114]:
data_dict = json.loads(json_objects)
json_file_path = "sample.json"
with open(json_file_path, 'w') as json_file:
    json.dump(data_dict, json_file,indent=2)

In [99]:
with open('sample.json', 'r') as openfile:

	# Reading from json file
	json_object = json.load(openfile)


print(json_object)
print(type(json_object))

{'filename': 'breast-cancer-wisconsin.csv', 'model_name': 'decision_tree', 'decision_tree_parameters': {'max_depth': 3, 'splitter': 'random', 'criterion': 'log_loss'}, 'target_variable': 'Class', 'split': 0.2}
<class 'dict'>


In [ ]:


# Read the JSON file
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Modify the value (change from str to int)
data[""] = int(data["example_key"])

# Write the modified data back to the JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=2)  # indent for pretty formatting (optional)


In [32]:
def to_numeric(F6):
    if F6 == "?":
        return np.nan
    else:
        return int(F6)

def file_preprocess(filename):
  df = pd.read_csv(filename)
  df['F6'] = df['F6'].apply(to_numeric)
  mean_F6 = df['F6'].mean()
  df['F6'] = df['F6'].fillna(mean_F6)
  return df

In [115]:
def runner(json_file):
  # Load parameters from JSON
  with open(json_file, 'r') as file:
      parameters = json.load(file)

  model_name = parameters['model_name']
  df = file_preprocess(parameters['filename'])

  #Check for target_variable is present or not
  target_variable = parameters.get("target_variable", None)
  if target_variable is None:
      raise ValueError("Target variable not specified in the parameters.")

  X = df.drop(columns=[target_variable])
  y = df[target_variable]

  # Define default parameters for SVMClassifier
  default_lr_parameters = {
      "penalty": 'l2',
      "dual": False,
      "tol": 0.0001,
      "C": 1.0,
      "fit_intercept": True,
      "intercept_scaling": 1,
      "class_weight": None,
      "random_state": None,
      "solver": 'lbfgs',
      "max_iter": 100,
      "multi_class": 'auto',
      "verbose": 0,
      "warm_start": False,
      "n_jobs": None,
      "l1_ratio": None
  }
  default_svm_parameters = {
    'C': 1.0,
    'kernel': 'rbf',
    'degree': 3,
    'gamma': 'scale',
    'coef0': 0.0,
    'shrinking': True,
    'probability': False,
    'tol': 0.001,
    'cache_size': 200,
    'class_weight': None,
    'verbose': False,
    'max_iter': -1,
    'decision_function_shape': 'ovr',
    'break_ties': False,
    'random_state': None
  }
  default_decision_tree_parameters = {
    "criterion": "gini",
    "splitter": "best",
    "max_depth": None,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
    "min_weight_fraction_leaf": 0.0,
    "max_features": None,
    "random_state": None,
    "max_leaf_nodes": None,
    "min_impurity_decrease": 0.0,
    "class_weight": None,
    "ccp_alpha": 0.0
 }

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=parameters['split'], random_state=42)

  def_param = {
      "decision_tree" : {"param_dict" : "default_decision_tree_parameters", "lib_name" : "DecisionTreeClassifier"},
      "svm" : {"param_dict" : "default_decision_tree_parameters", "lib_name" : "DecisionTreeClassifier"},
      "lr" : {"param_dict" : "default_decision_tree_parameters", "lib_name" : "LogisticRegression"}
  }

  param = def_param[model_name]["param_dict"]
  lib_name = def_param[model_name]["lib_name"]
  # print(param, lib_name)

  # Merge default and user-provided parameters
  merged_parameters = {**eval(param), **parameters.get("param", {})}
  # print(merged_parameters)

  # Initialize the Decision Tree model with the merged parameters
  model = eval(lib_name)(**merged_parameters)

  # Train the Decision Tree model
  model.fit(X_train, y_train)

  # Make predictions on the test set
  y_pred = model.predict(X_test)

  # Evaluate the model
  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {accuracy}")
  print(merged_parameters)
  print(param)
  print(eval(param))
  print(parameters["param"])

In [116]:
runner("sample.json")

Accuracy: 0.95
{'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': None, 'random_state': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'class_weight': None, 'ccp_alpha': 0.0}
default_decision_tree_parameters
{'criterion': 'gini', 'splitter': 'best', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': None, 'random_state': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'class_weight': None, 'ccp_alpha': 0.0}
{'max_depth': 3, 'splitter': 'random', 'criterion': 'log_loss'}


In [28]:
data = {"model_reply": json_objects}

with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)

In [46]:
import json

# Your dictionary
my_dict = {
    "name": "John",
    "age": 30,
    "city": "New York"
}

# Specify the file path
json_file_path = "sample.json"

# Write the dictionary to the JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(my_dict, json_file, indent=2)  # indent for pretty formatting (optional)
